# Solving a max-cut problem using QAOA

## What is max-cut problem?

Based on the [Wikipedia page](https://en.wikipedia.org/wiki/Maximum_cut), a maximum cut of a graph is a cut whose size is at least the size of any other cut. In other words, a maximum cut divides the vertices into two complementary sets $S$ and $T$, such that the number of edges between the two sets is as large as possible. 

## Approximating using Ising model

The Max Cut problem is equivalent to minimizing the Hamiltonian of a spin glass model, most simply the Ising model. The Hamiltonian for an Ising model on a graph $G$ and only nearest-neighbor interactions is  
$$
H[s]=-\sum_{i,j\in E(G)}J_{ij}s_is_j
$$  
Each vertex $i$ of the graph is a spin state that can take value $\pm1$. A spin configuration will split a graph $V(G)$ into to sets, those with spin up $V^+$ and those with spin down $V^-$. The Hamiltonian can be written as  
$$
\begin{align}
H[s] = -\sum_{ij\in E(V^+)}J_{ij}-\sum_{ij\in E(V^-)}J_{ij}+\sum_{ij\in\delta(V^+)}J_{ij}  \\ =-\sum_{ij\in E(G)}J_{ij}+2\sum_{ij\in\delta(V^+)}J_{ij}  \\ =C+2\sum_{ij\in\delta(V^+)}J_{ij}
\end{align}
$$  
where $\delta(V^+)$ is the set of edges that connects the two sets ($s_is_j=-1$).  
Minimizing this Hamiltonian is equivalent to the min-cut problem. It’s equivalent to the max-cut problem if we set the graph weights as $w_{ij}=-J_{ij}$.

## What is QAOA?

Quantum Approximate Optimization Algorithm (QAOA), is a method for solving combinatorial optimization problems on NISQ devices. It can be used to find approximate solutions to problems that can be phrased as finding the optimal bitstring. The steps of this algorithm can be described as follows (taken from [PennyLane Demo "Intro to QAOA"](https://pennylane.ai/qml/demos/tutorial_qaoa_intro.html)):

1. Define a cost Hamiltonian $H_C$. We should be able to obtain the solution to the optimization problem from its ground state. 
2. Define a mixer Hamiltonian $H_M$. 
3. Construct circuits $e^{-i\gamma H_C}$ and $e^{-i\alpha H_M}$.
4. Choose a parameter $n\geq1$, and building $n$ layers of circuit $U(\boldsymbol\gamma, \boldsymbol\alpha)=e^{-i\alpha_nH_M}e^{-i\gamma_nH_C}...e^{-i\alpha_1H_M}e^{-i\gamma_1H_C}$.
5. Prepare an initial state, apply $U(\boldsymbol\gamma, \boldsymbol\alpha)$, use classical techniques to optimize the parameters.
6. Approximate solutions is obtained by measuring the final output state.

## Apply QAOA to the problem

It turns out that the max-cut problem is demonstrated [in the Qiskit textbook](https://qiskit.org/textbook/ch-applications/qaoa.html).
First, the Hamiltonians. The problem hamiltonian is  
$$
H_P = {1\over2}\sum_{i,j\in E(G)}Z_i\otimes Z_j
$$  
and the mixing Hamiltonian is  
$$
H_M = \sum_{i\in V(G)}X_i
$$  
where $X$ is the PauliX operator.  

In the algorithm, the unitary operators we are actually applying are $e^{-i\alpha H_M}$ and $e^{-i\gamma H_P}$. They can both be broken down into individual parts as  
$$
\begin{align}
U(H_M)&=e^{-i\alpha H_M}=\prod_{i\in V(G)}e^{-i\alpha X_i}  \\ U(H_P)&=e^{-i\gamma H_P}=\prod_{i,j\in E(G)}e^{-i\gamma Z_iZ_j}
\end{align}
$$  
Each term in $U(H_M)$ corresponds to a X-rotation, and each term in $U(H_P)$ corresponds to a rotation about ZZ. In Qiskit, we can use the RX gate and the RZZ gate to implement these operators.

In [1]:
# import necessary libraries

# For graph and plotting
import networkx as nx
import matplotlib.pyplot as plt

# For quantum circuits
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram

# For optimization
from scipy.optimize import minimize


/Users/YaqiHan/opt/anaconda3/lib/python3.8/site-packages/qiskit/validation/fields/custom.py:76: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  numpy.integer, numpy.float,
/Users/YaqiHan/opt/anaconda3/lib/python3.8/site-packages/qiskit/quantum_info/operators/pauli.py:111: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  z = np.zeros(len(label), dty